# 誤り訂正符号の実装

このノートブックでは、基本的な誤り訂正符号であるShorの符号について、Qiskitのシミュレータを用いて実装を試みる。   
実装した回路を元に、エラーが入った際の挙動を確認する。

---
# ライブラリのインポート

In [ ]:
%pip install qiskit==0.45.2 qiskit-aer==0.12.0
%pip install pylatexenc

In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit_aer.noise import pauli_error

# エラーチャネルの作成

Qiskitの機能を利用して、エラーが起こる部分を作成します。

ここでは、一定確率でビット反転エラーと位相反転エラーが起こるようなものを作成します。

In [ ]:
p_error = 0.02 # エラー確率

In [ ]:
def make_bitphase_error_channel(p_error: float, print_flag: bool = True) -> QuantumChannelInstruction:
    bit_flip = pauli_error([('X', p_error), ('I', 1 - p_error)])
    phase_flip = pauli_error([('Z', p_error), ('I', 1 - p_error)])

    bitphase_flip = bit_flip.compose(phase_flip)
    
    if print_flag:
        print(bitphase_flip)
    return bitphase_flip.to_instruction()

In [ ]:
bitphase_flip = make_bitphase_error_channel(p_error)

それぞれのエラーは独立に発生するため、上記の出力のような回路が確率的に適用されることと等価です。

# 単純な回路で試す

Xゲートを適用するだけの回路で試してみます。

In [ ]:
n_shots = 10_000 # シミュレーターでのサンプリング回数
backend_sim = AerSimulator() # シミュレーターの用意

In [ ]:
# エラーなしの回路
n_qubits = 1
circ_ideal = QuantumCircuit(n_qubits, 1)

circ_ideal.x(0)

circ_ideal.measure(0, 0)
circ_ideal.draw("mpl")

In [ ]:
result_ideal = backend_sim.run(circ_ideal, shots=n_shots).result()
plot_histogram(result_ideal.get_counts(0))

次に、Xゲート適用後にエラーが発生するようにします。

In [ ]:
# エラーありの回路
n_qubits = 1
circ_noise = QuantumCircuit(n_qubits, 1)

circ_noise.x(0)

circ_noise.append(bitphase_flip, [0])

circ_noise.measure(0, 0)
circ_noise.draw("mpl")

In [ ]:
result_noise = backend_sim.run(circ_noise, shots=n_shots).result()
plot_histogram(result_noise.get_counts(0))

本来出現しないはずの0が約2％の確率で出現しています。  
（今回のケースでは、位相反転エラーの影響はサンプリングの結果では見えないです）

# ビット反転エラーを検出する回路

まずは、3量子ビットのビット反転エラーを検出する回路を実装します。

In [ ]:
def make_circ(noise_channel: list[int] = [], p_error: float = 0.02) -> QuantumCircuit:
    # noise_channel : ノイズをかけたいチャネル(int)を入れたリスト
    # p_error : エラーの発生確率
    
    # エラーの定義
    bit_flip = pauli_error([('X', p_error), ('I', 1 - p_error)]).to_instruction()
    
    # 回路の記述
    circ = QuantumCircuit(3, 1)
    
    circ.x(0)
    circ.barrier()
    
    # 符号化
    circ.cx(0, 1)
    circ.cx(0, 2)
    
    circ.barrier()
    
    # エラーが発生する部分
    for i in noise_channel:
        assert (0 <= i) and (i < 3)
        circ.append(bit_flip, [i]) # i番目の量子ビットに確率的にビット反転を発生
    
    circ.barrier()
    
    # 復号
    circ.cx(0, 2)
    circ.cx(0, 1)
    circ.ccx(1, 2, 0)
    
    circ.measure(0, 0)
    
    return circ

まずはエラーなしで動かしてみます。

In [ ]:
circ_ideal = make_circ()

In [ ]:
circ_ideal.draw("mpl") # 回路を描画する

In [ ]:
result_ideal = backend_sim.run(circ_ideal, shots=n_shots).result()
plot_histogram(result_ideal.get_counts(0))

次に、1つの量子ビットにエラーをかけてみます。

In [ ]:
circ_noise = make_circ(noise_channel=[0]) # 符号化後、0番の量子ビットに確率的にエラーを発生させる

circ_noise.draw("mpl") # 回路を描画する

In [ ]:
result_noise = backend_sim.run(circ_noise, shots=n_shots).result()
plot_histogram(result_noise.get_counts(0))

一つのチャネルにエラーをかけても、問題なく、すべての出力が1になりました。

# 課題1

複数のチャネルにエラーをかけてみて、誤り訂正が成功するかどうかを確認してください。

失敗する場合、エラー率がどの程度になるのかも確認してみてください。

---

エラーが3箇所に発生する場合でも、エラーこそ発生しますが、誤り訂正を実装する前に比べると、極めて低確率になります。

# Shorの符号

Shorの符号は9量子ビットを用いることで、1量子ビットまでのビット反転エラーと位相反転エラーの双方に対応できる符号です。  

より少ない量子ビットで実現できる符号化方法も知られていますが、まずは理解のしやすいShorの符号から実装してみます。

In [ ]:
def make_shor_code(noise_channel : list[int] = [], p_error: float = 0.02) -> QuantumCircuit:
    # noise_channel : ノイズをかけたいチャネル(int)を入れたリスト
    # p_error : エラーの発生確率
    
    # エラーの定義
    bitphase_flip = make_bitphase_error_channel(p_error, print_flag=False)
    
    # 回路の記述
    n_qubits = 9
    circ = QuantumCircuit(n_qubits, 1)
   
    circ.x(0)
    circ.barrier()
    
    # 符号化
    circ.cx(0, 3)
    circ.cx(0, 6)

    circ.h(0)
    circ.h(3)
    circ.h(6)

    circ.cx(0, 1)
    circ.cx(0, 2)

    circ.cx(3, 4)
    circ.cx(3, 5)

    circ.cx(6, 7)
    circ.cx(6, 8)


    circ.barrier()

    # エラーが発生する部分
    for i in noise_channel:
        assert (0 <= i) and (i < 9)
        circ.append(bitphase_flip, [i])

    circ.barrier()


    # 復号
    circ.cx(6, 8)
    circ.cx(6, 7)

    circ.cx(3, 5)
    circ.cx(3, 4)

    circ.cx(0, 2)
    circ.cx(0, 1)

    circ.ccx(2, 1, 0)
    circ.ccx(5, 4, 3)
    circ.ccx(8, 7, 6)

    circ.h(0)
    circ.h(3)
    circ.h(6)

    circ.cx(0, 6)
    circ.cx(0, 3)
    circ.ccx(6, 3, 0)

    circ.measure([0], [0])
    
    return circ

まずは、エラーをかけない状態で実行してみて、本来の状態が得られることを確認します。

In [ ]:
circ_ideal = make_shor_code()

In [ ]:
circ_ideal.draw("mpl")

In [ ]:
result_ideal = backend_sim.run(circ_ideal, shots=n_shots).result()
plot_histogram(result_ideal.get_counts(0))

問題なく元の状態が得られました。

# 課題2

複数のチャネルにエラーをかけてみて、誤り訂正が成功するかどうかを確認してください。  
失敗する場合、エラー率がどの程度になるのかも確認してみてください。

# 課題3

エラー発生確率を変化させて、Shorの符号による誤り訂正の効果を確認してみましょう。  
ここでは、すべての量子ビットでエラーが発生するようにします。